simplest case of NN

In [8]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

import torch

import torch.nn as nn
import torch.optim as optim

logistic regression

In [9]:
# Simulate data for logistic regression with two regressors

n = 100

x1 = np.random.normal(0, 1, n)
x2 = np.random.normal(0, 1, n)

# True parameters
beta_0 = 0.0  # intercept
beta_1 = 1.0  # coefficient for x1
beta_2 = -1.0  # coefficient for x2

# Compute linear predictor and probability using the logistic function
lin_pred = beta_0 + beta_1 * x1 + beta_2 * x2
p = 1 / (1 + np.exp(-lin_pred))

# Generate binary outcome (y) based on the computed probabilities
y = np.random.binomial(1, p, n)

# Create a DataFrame with the simulated data
logit_data = pd.DataFrame({'x1': x1, 'x2': x2, 'y': y})

In [10]:
# Convert our logistic regression data (logit_data) to PyTorch tensors
X_logit_tensor = torch.tensor(logit_data[['x1', 'x2']].values, dtype=torch.float32)
y_logit_tensor = torch.tensor(logit_data['y'].values.reshape(-1, 1), dtype=torch.float32)


In [11]:
# Define a logistic regression model as a neural network with one hidden layer
class LogisticRegressionModel(nn.Module):
    def __init__(self, weight_init=None, bias_init=None):
        super(LogisticRegressionModel, self).__init__()
        self.hidden = nn.Linear(2, 1)
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        x = self.hidden(x)
        x = self.activation(x)
        return x

model = LogisticRegressionModel()

In [12]:
# Define the loss function and the optimizer
criterion = nn.BCELoss() # binary cross-entropy loss
# notice: AI suggested "BCELogitsLoss", which was wrong.

optimizer = optim.Adam(model.parameters(), lr=0.01)


In [13]:

# Train the model
epochs = 1000  # you can adjust epochs for convergence
for epoch in range(epochs):
    optimizer.zero_grad()
    logits = model(X_logit_tensor)  # raw scores
    loss = criterion(logits, y_logit_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 200 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 200/1000, Loss: 0.5143
Epoch 400/1000, Loss: 0.4970
Epoch 600/1000, Loss: 0.4961
Epoch 800/1000, Loss: 0.4961
Epoch 1000/1000, Loss: 0.4961


In [14]:

# Report the estimated weights (the weights and bias of the one-layer NN)
print("\nEstimated weights (input layer parameters):")
for name, param in model.named_parameters():
    print(f"{name}: {param.data.numpy()}")

# Compute final log loss on the training data
# with torch.no_grad():
final_loss = n * criterion(model(X_logit_tensor), y_logit_tensor).item()
print(f"\nFinal LogLoss: {final_loss:.4f}")


Estimated weights (input layer parameters):
hidden.weight: [[ 1.1459869 -1.3353062]]
hidden.bias: [-0.02284491]

Final LogLoss: 49.6062


In [15]:
# Add a constant to the regressors for the intercept
X_logit = sm.add_constant(logit_data[['x1', 'x2']])

# Fit the logistic regression model using statsmodels
logit_model = sm.Logit(logit_data['y'], X_logit).fit()
# print("Logistic Regression Results Summary:")
# print(logit_model.summary())

# Display the true parameters and the estimated coefficients
print("\nTrue parameters:")
print(f"Intercept: {beta_0}, x1: {beta_1}, x2: {beta_2}")
print("\nEstimated parameters:")
print(logit_model.params)
# print the value of the log-likelihood function
print(f"\nLog-likelihood: {-logit_model.llf:.4f}")

Optimization terminated successfully.
         Current function value: 0.496062
         Iterations 6

True parameters:
Intercept: 0.0, x1: 1.0, x2: -1.0

Estimated parameters:
const   -0.022843
x1       1.146098
x2      -1.335520
dtype: float64

Log-likelihood: 49.6062
